In [1]:
from pysentimiento import *
import pandas as pd
import re #sirve para concatenar texto
import nltk #procesamiento de palabras
from nltk.corpus import stopwords
from pysentimiento.preprocessing import preprocess_tweet

In [2]:
df = pd.read_csv("tuits_bayes.csv", encoding ="latin-1")
df.head(10) #


,status_id,screen_name,text
0,832421505778618368,lopezobrador_,Josefina Vázquez Mota debe informar qué hizo c...
1,833130565087985664,lopezobrador_,Ya ni la burla perdonan: bajaron 2 centavos la...
2,833501308548177921,lopezobrador_,Martín Moreno no votará por mí. Comprendo. Es ...
3,833895239211364357,lopezobrador_,En Chicago dije que iremos a Nueva York (ONU) ...
4,834183763923841024,lopezobrador_,Entérate y apoya con tu firma la denuncia cont...
5,834565985583525888,lopezobrador_,"Qué república ni qué ocho cuartos, es la monar..."
6,835297483521564672,lopezobrador_,"Salieron a defender a Yunes Linares, Calderón,..."
7,835903628388990976,lopezobrador_,"En 2010, la trasnacional Odebrecht entregó sob..."
8,836417496420925440,lopezobrador_,Calderón dice donará su sueldo como expresiden...
9,836745914022682624,lopezobrador_,¡Las pensiones de los expresidentes de México ...


In [3]:
#df = df.rename(columns={'Comentario':'text'})

In [4]:
black_list = ['con']
def cleaner(word): 
    word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', str(word), flags=re.MULTILINE)
    word = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', "", str(word))
    word = re.sub(r'ee.uu', 'eeuu', str(word))
    word = re.sub(r'\#\.', '', str(word))
    word = re.sub(r'\n', '', str(word))
    word = re.sub(r',', '', str(word))
    word = re.sub(r'\-', ' ', str(word))
    word = re.sub(r'\.{3}', ' ', str(word))
    word = re.sub(r'a{2,}', 'a', str(word))
    word = re.sub(r'é{2,}', 'é', str(word))
    word = re.sub(r'i{2,}', 'i', str(word))
    word = re.sub(r'ja{2,}', 'ja', str(word)) 
    word = re.sub(r'á', 'a', str(word))
    word = re.sub(r'é', 'e', str(word))
    word = re.sub(r'í', 'i', str(word))
    word = re.sub(r'ó', 'o', str(word))
    word = re.sub(r'ú', 'u', str(word))  
    word = re.sub('[^a-zA-Z]', ' ', str(word))
    word = " ".join([i for i in word.split() if i not in stopwords])
    word = preprocess_tweet(str(word))
    return word

In [5]:
len(df.text.to_list())

1349

In [6]:
stop = set(stopwords.words('spanish'))
additional_stopwords=set(black_list)
stopwords = stop.union(additional_stopwords)

In [7]:
%%time
df['text'] = df['text'].apply(cleaner)

CPU times: user 69.9 ms, sys: 1.53 ms, total: 71.4 ms
Wall time: 70.5 ms


In [8]:
df

,status_id,screen_name,text
0,832421505778618368,lopezobrador_,Josefina Vazquez Mota debe informar hizo mil m...
1,833130565087985664,lopezobrador_,Ya burla perdonan bajaron centavos gasolina En...
2,833501308548177921,lopezobrador_,Martin Moreno votara Comprendo Es mal escritor...
3,833895239211364357,lopezobrador_,En Chicago dije iremos Nueva York ONU Washingt...
4,834183763923841024,lopezobrador_,Enterate apoya firma denuncia ordenes Donald T...
...,...,...,...
1344,983475079744049152,DafneFCh,Hay cosas puede hacer Y simplemente rinde
1345,983475080507338753,09osuna,Ser feliz implica aceptar limitaciones
1346,983475080566136832,louder_thancat,Nada parece
1347,983475082570772485,lilyindescai,Cuando empiezas acordar cosas hiciste ebriedad


In [9]:
analyzer = create_analyzer(task="sentiment", lang="es")
emotion_analyzer = create_analyzer(task="emotion", lang="en")

loading configuration file config.json from cache at /Users/ff_didier/.cache/huggingface/hub/models--finiteautomata--bertweet-base-emotion-analysis/snapshots/64046df9cc41eab40e1ecde7d2b7fb42b971be5b/config.json
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sadness": 2,
    "surprise": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130

In [10]:
def process(row):
    res1 = analyzer.predict(row)
    res2 = emotion_analyzer.predict(row)
    return res1.output,res2.output


In [11]:
df['senti_puntuacion','Emotion'] = df['text'].apply(process)

In [12]:
df

,status_id,screen_name,text,"(senti_puntuacion, Emotion)"
0,832421505778618368,lopezobrador_,Josefina Vazquez Mota debe informar hizo mil m...,"(NEG, others)"
1,833130565087985664,lopezobrador_,Ya burla perdonan bajaron centavos gasolina En...,"(NEG, others)"
2,833501308548177921,lopezobrador_,Martin Moreno votara Comprendo Es mal escritor...,"(NEG, others)"
3,833895239211364357,lopezobrador_,En Chicago dije iremos Nueva York ONU Washingt...,"(NEU, others)"
4,834183763923841024,lopezobrador_,Enterate apoya firma denuncia ordenes Donald T...,"(NEU, others)"
...,...,...,...,...
1344,983475079744049152,DafneFCh,Hay cosas puede hacer Y simplemente rinde,"(NEU, others)"
1345,983475080507338753,09osuna,Ser feliz implica aceptar limitaciones,"(NEU, others)"
1346,983475080566136832,louder_thancat,Nada parece,"(NEU, others)"
1347,983475082570772485,lilyindescai,Cuando empiezas acordar cosas hiciste ebriedad,"(NEG, others)"


In [13]:
df.to_excel (r'sentimientoene.xlsx', index = False, header=True)


In [14]:
from pysentimiento.preprocessing import preprocess_tweet

# Replaces user handles and URLs by special tokens
preprocess_tweet("@perezjotaeme debería cambiar esto http://bit.ly/sarasa") # "@usuario debería cambiar esto url"

# Shortens repeated characters
preprocess_tweet("no entiendo naaaaaaaadaaaaaaaa", shorten=2) # "no entiendo naadaa"

# Normalizes laughters
preprocess_tweet("jajajajaajjajaajajaja no lo puedo creer ajajaj") # "jaja no lo puedo creer jaja"

# Handles hashtags
preprocess_tweet("esto es #UnaGenialidad")
# "esto es una genialidad"

'esto es hashtag una genialidad'

In [22]:
# Tarea - Didier Irias
import nltk

# Create bigrams
df['bigrams'] = df['text'].apply(lambda x: list(nltk.bigrams(x.split())))

# Print the first 5 rows
print(df['bigrams'])



0       [(Josefina, Vazquez), (Vazquez, Mota), (Mota, ...
1       [(Ya, burla), (burla, perdonan), (perdonan, ba...
2       [(Martin, Moreno), (Moreno, votara), (votara, ...
3       [(En, Chicago), (Chicago, dije), (dije, iremos...
4       [(Enterate, apoya), (apoya, firma), (firma, de...
                              ...                        
1344    [(Hay, cosas), (cosas, puede), (puede, hacer),...
1345    [(Ser, feliz), (feliz, implica), (implica, ace...
1346                                     [(Nada, parece)]
1347    [(Cuando, empiezas), (empiezas, acordar), (aco...
1348                                  [(Fulvio, Martins)]
Name: bigrams, Length: 1349, dtype: object
